In [1]:
%matplotlib inline
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import math
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In /home/paul/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/paul/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/paul/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
X = np.loadtxt('xtrain_challenge.csv', delimiter=',', skiprows = 1)
Y = np.loadtxt('ytrain_challenge.csv', delimiter=',', skiprows = 1)


In [34]:
x2=np.zeros(37)
for i in range(0,13): 
    x2[i]=X[0][13+i]
for i in range(13,26): 
    x2[i]=X[0][13-i]
for i in range(26,37):
    x2[i]=X[0][i]



[ 1.0000e+00  0.0000e+00  0.0000e+00 -2.9900e+00 -1.6610e+01  1.9100e+00
  1.6000e-01  8.1000e-01  3.8000e-01  1.7785e+02  3.0020e+01  0.0000e+00
  0.0000e+00]
[ 1.0000e+00  0.0000e+00  0.0000e+00 -2.9900e+00 -1.6610e+01  1.9100e+00
  1.6000e-01  8.1000e-01  3.8000e-01  1.7785e+02  3.0020e+01  0.0000e+00
  0.0000e+00]


In [45]:
new_X=[]
new_Y=[]
for i in range(len(X)):
    x1=X[i]
    x2=np.zeros(37)
    for i in range(0,13): 
        x2[i]=X[0][13+i]
    for i in range(13,26): 
        x2[i]=X[0][13-i]
    for i in range(26,37):
        x2[i]=X[0][i]
    new_X.append(x1)
    new_X.append(x2)
    new_Y.append(Y[i])
    new_Y.append(Y[i])

new_X=np.array(new_X)
new_Y=np.array(new_Y)


In [46]:
np.save("X.npy",new_X,allow_pickle=True)

In [21]:
N=X.shape[0]
ordre=np.arange(0,N)
np.random.shuffle(ordre)

# Use as new variables
X=X[ordre[:]]
Y=Y[ordre[:]]


In [22]:
X_train, X_test ,y_train, y_test = train_test_split(X,Y, test_size=0.33, random_state=42
)

In [23]:
y_test.mean()

0.03670091631760005

In [24]:
y_train.mean()

0.03722881922958191

### KNN 

Commencer par les Knn 

Toujours la même question : sur quelles datas? 
Apparement, y'a que des 0 : faire de la data augmentation sur les 1 ? 
Dans tous les cas, va falloir équilibrer le nombre de 0 et de 1 un peu, quoique dans le test_set y'aura peut etre que des 1 aussi ? 
Trouver un moyen de donner une importance plus grande aux résultats sur les étiquettes 1 (poids) 

Tout est expliqué dans https://medium.com/@bluedme_tech/comment-traiter-les-probl%C3%A8mes-de-classification-d%C3%A9s%C3%A9quilibr%C3%A9e-en-machine-learning-8c3bc95ca25b

Faire un grid_search sur le paramètre "nombre de voisin" 

Dans tous les cas ça va pas être fou: les knn marchent pas bien quand il y a trop de features et là y'en a quand même 37 donc ça va pas le faire 
mais commencer par ça pour le mettre dans le rapport 



In [7]:
clf = KNeighborsClassifier(n_neighbors=10)
clf.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [65]:
clf.score(x_train,y_train)

0.99

In [66]:
clf.score(x_test,y_test)

0.994

In [103]:
clf.score(x_test[y_test==1],y_test[y_test==1])

0.8064516129032258

In [104]:
clf.score(x_test[y_test==0],y_test[y_test==0])

1.0

### MSE, modèle linéaire 

In [126]:
from sklearn.linear_model import LogisticRegression

lr= LogisticRegression(C=100,class_weight={0:1,1:20}).fit(X=x_train, y=y_train)


In [127]:
print(lr.score(x_train,y_train))
print(lr.score(x_test,y_test))
print(lr.score(x_test[y_test==1],y_test[y_test==1]))
print(lr.score(x_test[y_test==0],y_test[y_test==0]))

1.0
0.989
0.967741935483871
0.9896800825593395


c'est déjà mieux, faut jouer avec le paramètre de régularisation C, avec une échelle logarithmique 

### SVM

In [25]:
from sklearn.svm import LinearSVC

In [ ]:
svm=LinearSVC(C=0.1,class_weight={0:1,1:20}).fit(X_train,y_train)

In [125]:
print(svm.score(x_train,y_train))
print(svm.score(x_test,y_test))
print(svm.score(x_test[y_test==1],y_test[y_test==1]))
print(svm.score(x_test[y_test==0],y_test[y_test==0]))

0.998
0.988
0.967741935483871
0.9886480908152735


Ces deux modèles linéaires ça va être intéressant, peut-être pas obligés de faire du deep? 
Les modèles linéaires sont bons pour des très gros jeux de données, avec de grandes dimensions.

NOTES : 

Voyant que la grande majorité des observations appartient à la même catégorie, vous risquez de vous retrouver avec un modèle peu intelligent qui va toujours prédire la classe dominante.

https://medium.com/@bluedme_tech/comment-traiter-les-probl%C3%A8mes-de-classification-d%C3%A9s%C3%A9quilibr%C3%A9e-en-machine-learning-8c3bc95ca25b
VA BCP NOUS AIDER


Si j’écris par exemple class_weight = { 0 : 1, 1 : 5 } j’indique à l’algorithme que les valeurs positifs comptent 5 fois plus que les valeurs négatives. 

In [19]:
x_train = np.loadtxt('xtrain_challenge.csv', delimiter=',', skiprows = 1)
y_train = np.loadtxt('ytrain_challenge.csv', delimiter=',', skiprows = 1)

In [11]:
x_train.shape

(1068504, 37)

In [12]:


from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
model = Sequential()
model.add(Dense(20, input_dim=len(x_train[0]), activation='sigmoid'))
model.add(Dense(5, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))


In [22]:
y_train.mean()

0.03705461093266848

In [20]:
from keras import optimizers

# START CODE HERE

model.compile(loss='mean_squared_error',optimizer='sgd',metrics=['accuracy'])

In [27]:
epochs = 3

# START CODE HERE
history=model.fit(x_train, y_train, epochs=2, batch_size=10,verbose=2,class_weight={0:0.25,1:0.75})
# END CODE HERE

Epoch 1/2
 - 62s - loss: 0.0249 - accuracy: 0.9629
Epoch 2/2
 - 66s - loss: 0.0249 - accuracy: 0.9629


In [9]:
xtest = np.loadtxt('xtest_challenge.csv', delimiter=',', skiprows = 1)

In [33]:
ytest = model.predict(xtest)
print(ytest.shape)
np.savetxt('ytest_challenge_student.csv', ytest, fmt = '%1.0d', delimiter=',')

(3318296, 1)


In [7]:
from sklearn.neural_network import MLPClassifier

In [11]:
mean_on_train=X_train.mean(axis=0)
std_on_train=X_train.std(axis=0)
X_train_scaled=(X_train-mean_on_train)/std_on_train
X_test_scaled=(X_test-mean_on_train)/std_on_train
Xtest_scaled=(xtest-mean_on_train)/std_on_train

In [16]:
mlp=MLPClassifier(hidden_layer_sizes=(4,),verbose=1,max_iter=500,random_state=0,tol=10e-9,alpha=0.000001)

In [17]:
mlp.fit(X_train_scaled,y_train)

Iteration 1, loss = 0.10684054
Iteration 2, loss = 0.00981352
Iteration 3, loss = 0.00652484
Iteration 4, loss = 0.00598628
Iteration 5, loss = 0.00582978
Iteration 6, loss = 0.00580005
Iteration 7, loss = 0.00572768
Iteration 8, loss = 0.00570222
Iteration 9, loss = 0.00567621
Iteration 10, loss = 0.00564334
Iteration 11, loss = 0.00562268
Iteration 12, loss = 0.00563071
Iteration 13, loss = 0.00559822
Iteration 14, loss = 0.00559029
Iteration 15, loss = 0.00558162
Iteration 16, loss = 0.00557532
Iteration 17, loss = 0.00556854
Iteration 18, loss = 0.00557630
Iteration 19, loss = 0.00555087
Iteration 20, loss = 0.00554918
Iteration 21, loss = 0.00553837
Iteration 22, loss = 0.00555135
Iteration 23, loss = 0.00552160
Iteration 24, loss = 0.00552372
Iteration 25, loss = 0.00552043
Iteration 26, loss = 0.00553136
Iteration 27, loss = 0.00551842
Iteration 28, loss = 0.00550254
Iteration 29, loss = 0.00549020
Iteration 30, loss = 0.00552775
Iteration 31, loss = 0.00549065
Iteration 32, los

MLPClassifier(activation='relu', alpha=1e-06, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(4,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=0, shuffle=True, solver='adam',
              tol=1e-08, validation_fraction=0.1, verbose=1, warm_start=False)

In [18]:
mlp.score(X_test_scaled,y_test)

0.9982983888578503

In [15]:
mlp.score(X_test_scaled[y_test==1],y_test[y_test==1])

0.9714780252859723

In [150]:
ytest = mlp.predict(xtest_scaled)
print(ytest.mean())
np.savetxt('ytest_challenge_student2.csv', ytest, fmt = '%1.0d', delimiter=',')

0.034668094708850565


In [151]:
ytest.mean()

0.034668094708850565

In [56]:
from sklearn.model_selection import GridSearchCV, KFold

In [128]:
res=[]
for i in range(30):
    for j in range(30):
        res.append((i,j))
    res.append((i,))

param = {
    'hidden_layer_sizes': res}

param={'hidden_layer_sizes':tab}
model = MLPClassifier(tol=0.00001,max_iter=300)

grid = GridSearchCV(model, param, cv=5, n_jobs=-1)
grid.fit(x_train_scaled, y_train)

print('The best K is', grid.best_params_.get('hidden_layer_sizes'), ' with an average validation score equal to ', grid.best_score_)


# Let's now use the best model to assess the test score
BestModel=grid.best_estimator_
print('The test score is', BestModel.score(x_test_scaled, y_test))

The best K is 5  with an average validation score equal to  0.9981199999999999
The test score is 0.99815


In [113]:
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,),(12,),(2,),(15,15),(18,12)]}

In [ ]:
clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(x_train_scaled, y_train)
print('The best K is', grid.best_params_.get('hidden_layer_sizes'), ' with an average validation score equal to ', grid.best_score_)



In [9]:
import xgboost as xgb

mean_on_train=X.mean(axis=0)
std_on_train=X.std(axis=0)
X=(X-mean_on_train)/std_on_train
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [49]:
model = XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 0.4,
                      subsample = 0.8,
                      objective='binary:logistic', 
                      n_estimators=1000, 
                      reg_alpha = 0.3,
                      max_depth=4, 
                      gamma=10)

NameError: name 'XGBClassifier' is not defined

In [39]:
dtrain = xgb.DMatrix(X_train, label=Y_train)
dtest = xgb.DMatrix(X_test, label=Y_test)

In [40]:
param = {'max_depth': 30, 'eta': 1, 'objective': 'binary:logistic'}

In [41]:
evallist = [(dtest, 'eval'), (dtrain, 'train')]
num_round = 20
bst = xgb.train(param, dtrain, num_round, evallist)

[0]	eval-error:0.00252	train-error:0.00106
[1]	eval-error:0.00226	train-error:0.00062
[2]	eval-error:0.00195	train-error:0.00038
[3]	eval-error:0.00181	train-error:0.00026
[4]	eval-error:0.00175	train-error:0.00018
[5]	eval-error:0.00166	train-error:0.00012
[6]	eval-error:0.00164	train-error:0.00009
[7]	eval-error:0.00162	train-error:0.00007
[8]	eval-error:0.00153	train-error:0.00005
[9]	eval-error:0.00149	train-error:0.00003
[10]	eval-error:0.00144	train-error:0.00002
[11]	eval-error:0.00141	train-error:0.00001
[12]	eval-error:0.00142	train-error:0.00001
[13]	eval-error:0.00139	train-error:0.00000
[14]	eval-error:0.00138	train-error:0.00000
[15]	eval-error:0.00137	train-error:0.00000
[16]	eval-error:0.00137	train-error:0.00000
[17]	eval-error:0.00136	train-error:0.00000
[18]	eval-error:0.00135	train-error:0.00000
[19]	eval-error:0.00135	train-error:0.00000


In [42]:
d_ftest = xgb.DMatrix(xtest_scaled)
ytest = bst.predict(d_ftest)
print(ytest.mean())

0.034864794


In [48]:
ytest[ytest>0.5]=1
np.savetxt('ytest_challenge_student2.csv', ytest, fmt = '%1.0d', delimiter=',')

In [14]:
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from scipy.stats import uniform, randint

In [12]:
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


In [18]:
from xgboost.sklearn import XGBClassifier 
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV

xgb_model = xgb.XGBClassifier()

params = {
    #"colsample_bytree": uniform(0.7, 0.3),
    #"gamma": uniform(0, 0.5),
    #"learning_rate": uniform(0.05, 0.3), # default 0.1 
    "max_depth": randint(3, 7), # default 3
    #"n_estimators": randint(100, 150), # default 100
    #"subsample": uniform(0.6, 0.4)
}

search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=42, n_iter=1, cv=3, verbose=1, n_jobs=1, return_train_score=True)

search.fit(X, Y)

report_best_scores(search.cv_results_, 1)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  3.0min finished


Model with rank: 1
Mean validation score: 0.999 (std: 0.000)
Parameters: {'max_depth': 5}



In [61]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(xgbreg, params, n_jobs=1)  
gs.fit(X_train, Y_train)  
gs.best_model_  

KeyboardInterrupt: 

In [11]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(X, Y)

y_pred = xgb_model.predict(X)

print(abs(Y-y_pred).mean())

KeyboardInterrupt: 